In [1]:
DATA_NAME = 'coco-indoor' 
TRANSFORM = 'wavelet-horizontal'
CHANNEL = 'gray'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_gray_horizontal_wavelet_indoor_coco.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,2,3,4,5,6,7,8,9
2,5815.09,0.12,0.04,0.03,0.00,0.00,-0.00,-0.00
3,0.12,1326.06,-0.01,0.01,-0.00,0.00,-0.00,0.00
4,0.04,-0.01,292.65,0.00,0.00,-0.00,0.00,-0.00
5,0.03,0.01,0.00,55.13,0.00,0.00,0.00,-0.00
6,0.00,-0.00,0.00,0.00,10.02,0.00,0.00,0.00
7,0.00,0.00,-0.00,0.00,0.00,1.79,-0.00,0.00
8,-0.00,-0.00,0.00,0.00,0.00,-0.00,0.34,0.00
9,-0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,0.08


In [10]:
corr_matrix = cov_matrix / np.sqrt(np.outer(np.diag(cov_matrix), np.diag(cov_matrix)))
corr_df = pd.DataFrame(corr_matrix, index=GROUPS, columns=GROUPS)
corr_df.round(5)

,2,3,4,5,6,7,8,9
2,1.00000,0.00004,0.00003,0.00005,0.00001,0.00000,-0.00000,-0.00002
3,0.00004,1.00000,-0.00002,0.00004,-0.00000,0.00002,-0.00005,0.00009
4,0.00003,-0.00002,1.00000,0.00002,0.00001,-0.00006,0.00007,-0.00006
5,0.00005,0.00004,0.00002,1.00000,0.00002,0.00002,0.00002,-0.00003
6,0.00001,-0.00000,0.00001,0.00002,1.00000,0.00002,0.00003,0.00004
7,0.00000,0.00002,-0.00006,0.00002,0.00002,1.00000,-0.00004,0.00001
8,-0.00000,-0.00005,0.00007,0.00002,0.00003,-0.00004,1.00000,0.00003
9,-0.00002,0.00009,-0.00006,-0.00003,0.00004,0.00001,0.00003,1.00000


In [11]:
np.linalg.norm(cov_matrix - np.diag(np.diag(cov_matrix)))  # Remove diagonal elements for visualization

0.18613677996929953

In [12]:
pca = PCA()
pca.fit(cov_matrix)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[4.23181732e+06 2.15400878e+05 1.01960373e+04 3.47571729e+02
 1.07835741e+01 3.05643030e-01 8.78911976e-03 5.59879390e-34]

Principal components (eigenvectors):


,0,1,2,3,4,5,6,7
0,0.999387,-0.034261,-0.007132,-0.001336,-0.000244,-0.000043,-0.000008,-0.000002
1,0.033949,0.998643,-0.038856,-0.006907,-0.001256,-0.000223,-0.000044,-0.000009
2,0.008383,0.038282,0.998436,-0.039268,-0.006846,-0.001223,-0.000232,-0.000053
3,0.001882,0.008280,0.038587,0.998072,-0.047154,-0.008062,-0.001550,-0.000344
4,0.000427,0.001875,0.008501,0.046241,0.996936,-0.061402,-0.011342,-0.002498
5,0.000100,0.000439,0.001999,0.010588,0.059468,0.992999,-0.099340,-0.020758
6,0.000028,0.000122,0.000546,0.002908,0.016008,0.091625,0.971417,-0.218377
7,0.000013,0.000055,0.000254,0.001344,0.007384,0.041476,0.215288,0.975641


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(len(GROUPS)), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0006134556866620455,
 0.00135674103641803,
 0.0015643463895533838,
 0.0019278959337430956,
 0.003064295164693309,
 0.0070005373358157374,
 0.028582729499912807,
 0.024359482381213327]